<a href="https://colab.research.google.com/github/ravimashru/100-days-of-deep-learning/blob/master/docs/days/028_Using_Autodiff_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Autodiff in TensorFlow

Autodiff is used by TensorFlow to calculate gradients automatically.

For example, consider the function: $$f(x, y) = 3x^2 + 5xy$$

In [1]:
def f(x, y):
  return 3 * x ** 2 + 5 * x * y

Let us compute the gradient of the function at $x = 5, y = 10$ manually.

In [2]:
x, y = 5, 10
eps = 1e-10

# Gradient w.r.t. x
df_dx = (f(x + eps, y) - f(x, y)) / eps
print('df_dx: ', df_dx)

# Gradient w.r.t. y
df_dy = (f(x, y + eps) - f(x, y)) / eps
print('df_dy: ', df_dy)

df_dx:  80.00029083632398
df_dy:  24.999735614983365


Using autodiff, this can be done as follows:

In [3]:
import tensorflow as tf

x, y = tf.Variable(5.), tf.Variable(10.)
with tf.GradientTape() as tape:
  z = f(x, y)

gradients = tape.gradient(z, [x, y])

print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=80.0>, <tf.Tensor: shape=(), dtype=float32, numpy=25.0>]


Sometimes, there can be numerical difficulties due to floating-point precision errors:

In [8]:
def softplus(x):
  return tf.math.log(1 + tf.math.exp(x))

x = tf.Variable([100.])
with tf.GradientTape() as tape:
  z = softplus(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

To fix this, we can decorate a function whose derivative we are interested in using the `@tf.custom_gradient` decorator and make it return its normal output and a stable function to calculate the derivative. The function that calculates the derivative will receive the gradients backpropagated so far as a parameter.

In [9]:
@tf.custom_gradient
def stable_softplus(x):
  
  exp = tf.exp(x)
  
  def softplus_gradient(grad):
    # Using chain rule, multiple gradient so far with gradient of function
    # Gradient of softplus = 1 / (1 + 1 / exp(x))
    return grad * (1 / (1 + 1 / exp))

  return tf.math.log(1 + exp), softplus_gradient

x = tf.Variable([100.])
with tf.GradientTape() as tape:
  z = stable_softplus(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>]